In [16]:
%run src_runs/widget.py


Initiating project interface...
Project interface initiated successfully.


Script path : src_runs/research/factors/2_update_factors_fix.py
Script args : {'email': 0, 'param': '20110101,20151231'}
Script path : src_runs/research/factors/2_update_factors_fix.py
Script args : {'email': 0, 'param': '20160101,20191231'}
Script path : src_runs/research/factors/2_update_factors_fix.py
Script args : {'email': 0, 'param': '20200101,20231231'}
